# Test Run DINO with FracNet

In [38]:
# activated dependecies
import torch
import os
import torchvision
from torchvision.transforms import v2

In [39]:
cwd = os.getcwd()
print("Current directory:", cwd)

Current directory: c:\Users\Finn Kopmann\Dokumente\Studium\Master StaDs\semester 3\consulting\code\Consulting-Project-simple-Prediction-Model\Backbone\DINO


In [40]:
# initiate DINO model
REPO_DIR = "dinov3"
CHECKPOINT_PATH = "weights\dinov3_vits16_pretrain_lvd1689m-08c60483.pth"

# DINOv3 ViT models pretrained on web images
dinov3_vits16 = torch.hub.load(REPO_DIR, 'dinov3_vits16', source='local', backbone_weights=CHECKPOINT_PATH)

Ignored kwargs: {'backbone_weights': 'weights\\dinov3_vits16_pretrain_lvd1689m-08c60483.pth'}


In [41]:
# load image
img_path = r"test_image\tor-zum-32-marco-gruell-sv-werder-bremen-17-jubel-mit-der-mannschaft-ger-sv-werder-bremen-vs-1.jpg"
image = load_image(img_path)
print(f"Image loaded successfully")

Image loaded successfully


In [42]:
def make_transform(resize_size: int = 256):
    to_tensor = v2.ToImage()
    resize = v2.Resize((resize_size, resize_size), antialias=True)
    to_float = v2.ToDtype(torch.float32, scale=True)
    normalize = v2.Normalize(
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225),
    )
    return v2.Compose([to_tensor, resize, to_float, normalize])

In [47]:
# test
batch = make_transform()(image).unsqueeze(0)

In [48]:
with torch.inference_mode():
    feats = dinov3_vits16.forward_features(batch) 

if isinstance(feats, dict):
    pooled_output = feats["x_norm_clstoken"]
    patch_tokens = feats["x_norm_patchtokens"]
else:
    # fallback if forward_features() not implemented
    tokens = model(batch) 
    pooled_output = tokens[:,0]          
    patch_tokens = tokens[:,1+model.num_register_tokens:,:]

In [51]:
pooled_output.shape

torch.Size([1, 384])